# purpose
the purpose of this notebook is to split off my locations so that they can be 

 1. reverse geocoded to zipcodes 
 2. augmented with demographic data
 

In [119]:
%config IPCompleter.greedy=True
import pandas as pd
pd.set_option('max_columns',30)
df = pd.read_csv('Chicago_Crimes_2008_to_2011.csv',skiprows =[1149093])

In [120]:
df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,388,4785,HP610824,10/07/2008 12:39:00 PM,000XX E 75TH ST,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,False,323,3.0,6.0,69.0,01A,1178207.0,1855308.0,2008,08/17/2015 03:03:40 PM,41.758276,-87.622451,"(41.758275857, -87.622451031)"
1,835,4786,HP616595,10/09/2008 03:30:00 AM,048XX W POLK ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,1533,15.0,24.0,25.0,01A,1144200.0,1895857.0,2008,08/17/2015 03:03:40 PM,41.870252,-87.746069,"(41.87025207, -87.746069362)"
2,1334,4787,HP616904,10/09/2008 08:35:00 AM,030XX W MANN DR,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,False,False,831,8.0,18.0,66.0,01A,1157314.0,1859778.0,2008,08/17/2015 03:03:40 PM,41.770990,-87.698901,"(41.770990476, -87.698901469)"
3,1907,4788,HP618616,10/10/2008 02:33:00 AM,052XX W CHICAGO AVE,0110,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,False,False,1524,15.0,37.0,25.0,01A,1141065.0,1904824.0,2008,08/17/2015 03:03:40 PM,41.894917,-87.757358,"(41.894916924, -87.757358147)"
4,2436,4789,HP619020,10/10/2008 12:50:00 PM,026XX S HOMAN AVE,0110,HOMICIDE,FIRST DEGREE MURDER,GARAGE,False,True,1032,10.0,22.0,30.0,01A,1154123.0,1886297.0,2008,08/17/2015 03:03:40 PM,41.843826,-87.709893,"(41.843826272, -87.709893465)"


In [121]:
locations = df['Location'].value_counts(dropna=False)
df_locations = locations.to_frame()
df_locations.reset_index(inplace =True)
df_locations.fillna(value='other',inplace=True)
df_locations.columns = ['locations','counts']
df_locations.head()

,locations,counts
0,other,28645
1,"(41.976290414, -87.905227221)",5710
2,"(41.754592961, -87.741528537)",3915
3,"(41.883500187, -87.627876698)",2468
4,"(41.909664252, -87.742728815)",2377


In [86]:
import geopy
import math
import numpy as np

points = []
for i, location in df_locations.locations.items():
    if location == 'other':
        points.append('other')
    else:
        point = geopy.Point(eval(location))
        points.append(point)
    
df_locations['points'] = points
df_locations.head()

,locations,counts,points
0,other,28645,other
1,"(41.976290414, -87.905227221)",5710,"41 58m 34.6455s N, 87 54m 18.818s W"
2,"(41.754592961, -87.741528537)",3915,"41 45m 16.5347s N, 87 44m 29.5027s W"
3,"(41.883500187, -87.627876698)",2468,"41 53m 0.600673s N, 87 37m 40.3561s W"
4,"(41.909664252, -87.742728815)",2377,"41 54m 34.7913s N, 87 44m 33.8237s W"


In [93]:

df_locations['tuples'] = df_locations.locations.apply(lambda x: x if 'other' else eval(x))
df_locations.head()

,locations,counts,points,tuples
0,other,28645,other,other
1,"(41.976290414, -87.905227221)",5710,"41 58m 34.6455s N, 87 54m 18.818s W","(41.976290414, -87.905227221)"
2,"(41.754592961, -87.741528537)",3915,"41 45m 16.5347s N, 87 44m 29.5027s W","(41.754592961, -87.741528537)"
3,"(41.883500187, -87.627876698)",2468,"41 53m 0.600673s N, 87 37m 40.3561s W","(41.883500187, -87.627876698)"
4,"(41.909664252, -87.742728815)",2377,"41 54m 34.7913s N, 87 44m 33.8237s W","(41.909664252, -87.742728815)"


In [101]:
df_locations.drop(columns='points').to_feather('locations')